<a href="https://colab.research.google.com/github/rushweigelt/rushweigelt.github.io/blob/master/TaT/src/models/Logisitic_Regression_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import seaborn as sb
from sklearn import metrics
from keras.models import Sequential, model_from_json
from keras.layers import Dense
from keras.utils import np_utils
from keras.regularizers import L1L2
import json
from sklearn_porter import Porter



In [0]:
url = 'https://raw.githubusercontent.com/rushweigelt/rushweigelt.github.io/master/TaT/data/combined_multi_bot_and_genuine_150.0k_split.csv'

In [3]:
pip install sklearn-porter

     |████████████████████████████████| 153kB 9.5MB/s 


In [0]:
#Function for test_train_split
def LRTestTrainSplit(data_path):
    data = pd.read_csv(data_path)
    data.fillna(0, inplace=True)
    x = data[['followerscount', 'friendscount', 'replycount', 'likecount', 'retweetcount', 'hashtagcount', 'urlcount', 'mentioncount']]
    y = data['label']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.35,  random_state=0)
    #client = Client(processes=False)
    lr = LogisticRegression()
    #with joblib.parallel_backend('dask'):
    lr.fit(x_train, y_train)

    x_test.fillna(x_test.mean())
    y_predict = lr.predict(x_test)
    '''
    #save attempt
    model_param = {}
    model_param['coef'] = list(lr.coef_)
    model_param['intercept'] = lr.intercept_.tolist()
    json_txt = json.dumps(model_param, indent=4)
    '''
    '''THIS MODEL ISNT SUPPORTED BY PORTER
    #save attempt 2
    porter = Porter(lr, language='java')
    output = porter.export(embed_data=True)
    print(output)
    '''
    with open('lr.param.txt', w) as file:
      file.write(json_text)
    confusion_matrix = pd.crosstab(y_test, y_predict, rownames=['Actual'], colnames=['Predicted'])
    print(confusion_matrix)
    matrix = sb.heatmap(confusion_matrix, annot=True)

    print('Accuracy: ', metrics.accuracy_score(y_test, y_predict))

In [0]:
def LR_KFold(data_path):
    data = pd.read_csv(data_path)
    data.fillna(0, inplace=True)
    x = data[['followerscount', 'friendscount', 'replycount', 'likecount', 'retweetcount', 'hashtagcount', 'urlcount', 'mentioncount']]
    y = data['label']
    scaler = MinMaxScaler(feature_range=(0,1))
    #x = scaler.fit_transform(x)
    #y = scaler.fit_transform(y)
    #kf = KFold(n_splits=10, shuffle=True)
    #kf.get_n_splits(x)
    #kf.get_n_splits(y)
    lr = LogisticRegression()
    cv_results = cross_validate(lr, x, y, cv=10)
    print(cv_results['test_score'])

In [0]:
def Keras_LR(data_path):
  #Get Data
  data = pd.read_csv(data_path)
  data.fillna(0, inplace=True)
  x = data[['followerscount', 'friendscount', 'replycount', 'likecount', 'retweetcount', 'hashtagcount', 'urlcount', 'mentioncount']]
  y = data['label']
  #map bot = 0, genuine = 1
  y = [0 if x=='bot' else x for x in y]
  y = [1 if x=='genuine' else x for x in y]
  print(y[-100:-1])
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.35,  random_state=0)
  #Build Model
  keras_LR_model = Build_Keras_Model()
  print(keras_LR_model.summary())
  #Fit Model
  keras_LR_model.fit(x_train, y_train, epochs=3, batch_size=5, verbose=1)
  #Save Model
  Save_Keras_Model(keras_LR_model)
  #Evaluate
  Evaluate_Keras_Model(keras_LR_model, x_test)



In [0]:
def Build_Keras_Model():
  model = Sequential()
  #Two becayse we have a binary categorizer
  model.add(Dense(2, activation='softmax', kernel_regularizer=L1L2(l1=0.0, l2=0.1), input_dim=8))
  model.add(Dense(1, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
def Save_Keras_Model(model):
    # saving model
    json_model = model.to_json()
    open('model_architecture.json', 'w').write(json_model)
    # saving weights
    model.save_weights('model_weights.h5', overwrite=True)

In [0]:
def Evaluate_Keras_Model(model, x_test_data):
  predictions = model.predict_classes(x_test_data, verbose=1)
  print(predictions)


In [8]:
LRTestTrainSplit(url)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (0,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neural_network.multilayer_perceptron module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should in

ValueError: ignored

In [0]:
Keras_LR(url)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (0,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 2)                 18        
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 3         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
195000/195000 [==============================] - 53s 271us/step - loss: 7.9672 - acc: 0.5004
Epoch 2/3
195000/195000 [==============================] - 52s 265us/step - loss: 7.9644 - acc: 0.5004
Epoch 3/3


In [0]:
def SaveModel(model):
  model.model.to_json